In [92]:
import os, fnmatch
import pandas as pd
pd.options.display.max_rows = 50

In [93]:
# Get the list of files
list_files = []
listOfFiles = os.listdir('data_idealo/.')  
pattern = "*.csv"  
for entry in listOfFiles:  
    if fnmatch.fnmatch(entry, pattern):
            list_files.append(entry)

In [94]:
# Put the dataframes in a dict
dict_files = {}
for file in list_files:
    dict_files[file] = pd.read_csv('data_idealo/' + file)
    
# Create the whole dataframe
df_idealo = pd.DataFrame(columns=["index_data", "product_name", "product_price", "currency", "voucher", "shop_name", "url_origin", "url_image", "id_page"])
for file in list_files:
    df_idealo = df_idealo.append(dict_files[file], ignore_index=True, sort=False)

In [95]:
# Process data
df_idealo = df_idealo.drop(df_idealo[df_idealo.currency.isna()].index) # Corrupted data
df_idealo.drop_duplicates(inplace=True) # Because a file got duplicates

df_idealo.id_page.fillna("", inplace=True)
df_idealo["id_page;"].fillna("", inplace=True)
df_idealo.id_page = df_idealo.id_page + df_idealo.loc[:, "id_page;"]
df_idealo.id_page = df_idealo.id_page.str.replace(';', '')

# Remove corrupted data
index_to_remove = df_idealo[df_idealo['product_name'].str.contains('document.getElementById', regex=False)].index
df_idealo.drop(index_to_remove, inplace=True)
df_idealo.drop(columns=['id_page;', 'Unnamed: 0'], inplace=True)

df_idealo.index_data = df_idealo.index_data.apply(lambda x: str(int(x)))

In [96]:
# Import df_train
df_train = pd.read_csv("../../Data/df_train_price_image.csv")
df_train.rename(columns={'productseller_name': 'product_name', 'Unnamed: 0':'index_data'}, inplace=True)
df_train.index_data = df_train.index_data.apply(lambda x: str(x))
df_train.label = df_train.label.apply(lambda x: str(int(x)))

In [97]:
# Merge the datasets
df_group_left = df_idealo.merge(df_train, left_on='index_data', right_on='index_data', how='left')

# Select the columns we need
df_group_left = df_group_left.loc[:, ['id_page', 'index_data', 'product_name_x', 't3_all_fr_prices_posweekly_prod.brand_seller',  'product_price', 'url_image', 'label']]
df_group_left.drop_duplicates(inplace=True)
df_group_left.rename(columns={'product_name_x': 'product_name', 't3_all_fr_prices_posweekly_prod.brand_seller': 'brand_seller'}, inplace=True)

In [98]:
# Merge the datasets
df_group_right = df_idealo.merge(df_train, left_on='index_data', right_on='index_data', how='right')

# Select the columns we need
df_group_right = df_group_right.loc[:, ['id_page', 'index_data', 'product_name_y', 't3_all_fr_prices_posweekly_prod.brand_seller', 'tbl11.price_avg', '_u1.product_image_url_seller', 'label']]
df_group_right.drop_duplicates(inplace=True)
df_group_right.loc[df_group_right.id_page.isna(), "id_page"] = df_group_right.id_page[df_group_right.id_page.isna()].index.values
df_group_right.rename(columns={'product_name_y': 'product_name', 'tbl11.price_avg': 'product_price', '_u1.product_image_url_seller': 'url_image', 't3_all_fr_prices_posweekly_prod.brand_seller': 'brand_seller'}, inplace=True)

In [99]:
# Concatenate 
df_group = pd.concat((df_group_left, df_group_right), axis=0, ignore_index=True)

In [100]:
# Remove duplicates (after selecting just a few columns we have duplicates)
df_group.drop_duplicates(inplace=True)

In [101]:
df_group.loc[:, ['id_page', 'index_data', 'product_name', 'brand_seller', 'product_price', 'url_image', 'label']].to_csv('df_metric_learning.csv', index=False)